# Titanic - Feature Engineering!
In this notebook, we will begin engineering features appropriate to build our machine learning model 

In [1]:
# Importing the necessary Python libraries
import pandas as pd
from category_encoders.one_hot import OneHotEncoder

In [2]:
# Loading in the training dataset
df_train = pd.read_csv('../data/raw/train.csv')

## What We Did Last Time...

In [3]:
# Dropping unnecessary columns
df_train.drop(columns = ['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace = True)

In [4]:
# Separating the supporting features (X) from the predictor feature (y)
X = df_train.drop(columns = ['Survived'])
y = df_train[['Survived']]

## Starting our Feature Engineering!

In [5]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       714 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB


## Engineering the Sex (Gender) Column

In [7]:
# Instantiating the One Hot Encoder object
sex_ohe_encoder = OneHotEncoder(use_cat_names = True, handle_unknown = 'ignore')

In [8]:
# Performing one hot encoding on the "Sex" column
sex_dummies = sex_ohe_encoder.fit_transform(X['Sex'])

/Users/dkhundley/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [9]:
# Viewing the newly generated "Sex" dummies
sex_dummies.head()

,Sex_male,Sex_female
0,1,0
1,0,1
2,0,1
3,0,1
4,1,0


In [10]:
# Joining the "Sex" dummies back into the overall dataset
X = pd.concat([X, sex_dummies], axis = 1)

In [11]:
# Dropping the original "Sex" column
X.drop(columns = ['Sex'], inplace = True)

## Engineering the Embarked Column

In [12]:
# Checking the value counts on the Embarked column
X['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [13]:
# Instantiating the One Hot Encoder object
embarked_ohe_encoder = OneHotEncoder(use_cat_names = True, handle_unknown = 'ignore')

In [14]:
# Showing which "Embarked" records from the raw dataset are null
X[X['Embarked'].isnull()]

,Pclass,Age,SibSp,Parch,Fare,Embarked,Sex_male,Sex_female
61,1,38.0,0,0,80.0,NaN,0,1
829,1,62.0,0,0,80.0,NaN,0,1


In [15]:
# Performing one hot encoding on the "Embarked" column
embarked_dummies = embarked_ohe_encoder.fit_transform(X['Embarked'])

/Users/dkhundley/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [16]:
# Viewing the "Embarked" dummies
embarked_dummies.tail()

,Embarked_S,Embarked_C,Embarked_Q,Embarked_nan
886,1,0,0,0
887,1,0,0,0
888,1,0,0,0
889,0,1,0,0
890,0,0,1,0


In [17]:
# Joining "Embarked" dummies back into X dataset
X = pd.concat([X, embarked_dummies], axis = 1)

In [18]:
# Getting rid of original "Embarked" column
X.drop(columns = ['Embarked'], inplace = True)

## Engineering the Age column

In [19]:
# Finding the median value of the "Age" column (which is 28.0)
median_age = X['Age'].median()

In [20]:
# Filling null values with the median age
X['Age'].fillna(median_age, inplace = True)

In [22]:
# Establishing our bins values and names
bin_labels = ['child', 'teen', 'young_adult', 'adult', 'elder']
bin_values = [-1, 12, 19, 30, 60, 100]

In [23]:
# Applying "Age" binning with Pandas cut
age_bins = pd.cut(X['Age'], bins = bin_values, labels = bin_labels)
df_age_bins = pd.DataFrame(age_bins)

In [24]:
# Viewing the distribution of age bins
df_age_bins['Age'].value_counts()

young_adult    422
adult          283
teen            95
child           69
elder           22
Name: Age, dtype: int64

In [25]:
# Instantiating the "Age" One Hot Encoder (OHE) object
age_ohe_encoder = OneHotEncoder(use_cat_names = True, handle_unknown = 'ignore')

In [26]:
# Fitting the age bins DataFrame to the OHE object
age_dummies = age_ohe_encoder.fit_transform(df_age_bins)

/Users/dkhundley/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [27]:
# Getting rid of the original "Age" (or actually, "age_imputed") column
X.drop(columns = ['Age'], inplace = True)

In [28]:
# Joining age_dummies back into full X dataset
X = pd.concat([X, age_dummies], axis = 1)

In [29]:
# Viewing the final engineered features
X.head()

,Pclass,SibSp,Parch,Fare,Sex_male,Sex_female,Embarked_S,Embarked_C,Embarked_Q,Embarked_nan,Age_child,Age_teen,Age_young_adult,Age_adult,Age_elder
0,3,1,0,7.2500,1,0,1,0,0,0,0,0,1,0,0
1,1,1,0,71.2833,0,1,0,1,0,0,0,0,0,1,0
2,3,0,0,7.9250,0,1,1,0,0,0,0,0,1,0,0
3,1,1,0,53.1000,0,1,1,0,0,0,0,0,0,1,0
4,3,0,0,8.0500,1,0,1,0,0,0,0,0,0,1,0


In [30]:
# Viewing information about the X DataFrame
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Pclass           891 non-null    int64  
 1   SibSp            891 non-null    int64  
 2   Parch            891 non-null    int64  
 3   Fare             891 non-null    float64
 4   Sex_male         891 non-null    int64  
 5   Sex_female       891 non-null    int64  
 6   Embarked_S       891 non-null    int64  
 7   Embarked_C       891 non-null    int64  
 8   Embarked_Q       891 non-null    int64  
 9   Embarked_nan     891 non-null    int64  
 10  Age_child        891 non-null    int64  
 11  Age_teen         891 non-null    int64  
 12  Age_young_adult  891 non-null    int64  
 13  Age_adult        891 non-null    int64  
 14  Age_elder        891 non-null    int64  
dtypes: float64(1), int64(14)
memory usage: 104.5 KB


In [33]:
# Saving cleaned dataset to disk
X.to_csv('../data/clean/X.csv', index = False)
y.to_csv('../data/clean/y.csv', index = False)